In [9]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\hazqe\.cache\huggingface\token
Login successful


In [10]:
from datasets import load_dataset

In [11]:
movie_dataset = load_dataset("datadrivenscience/movie-genre-prediction")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/54000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/hazqe/.cache/huggingface/datasets/datadrivenscience___parquet/datadrivenscience--movie-genre-prediction-01acd85570f2b187/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
movie_dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 54000
    })
})

In [15]:
movie_dataset["test"][0]

{'id': 16863,
 'movie_name': 'A Death Sentence',
 'synopsis': "12 y.o. Ida's dad'll die without a DKK1,500,000 operation. Ida plans to steal the money from the bank, her mom installed alarm systems in. She'll need her climbing skills, her 2 friends and 3 go-karts.",
 'genre': 'action'}

In [16]:
movie_dataset["train"][0]

{'id': 44978,
 'movie_name': 'Super Me',
 'synopsis': 'A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.',
 'genre': 'fantasy'}

In [21]:
set(movie_dataset["train"]["genre"])

{'action',
 'adventure',
 'crime',
 'family',
 'fantasy',
 'horror',
 'mystery',
 'romance',
 'scifi',
 'thriller'}

In [28]:
model_id = "prajjwal1/bert-medium"

In [29]:
from transformers import AutoTokenizer

model_tokenizer = AutoTokenizer.from_pretrained(model_id)

C:\Users\hazqe\AAAPython\Anaconda\envs\ai_triage\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hazqe\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [27]:
max_len_movie_name = 0
max_len_synopsis = 0
for i in range(len(movie_dataset["train"])):
    curr_len_movie_name = len(movie_dataset["train"][i]["movie_name"])
    if curr_len_movie_name > max_len_movie_name:
        max_len_movie_name = curr_len_movie_name
        
    curr_len_synopsis = len( movie_dataset["train"][i]["synopsis"])
    if curr_len_synopsis > max_len_synopsis:
        max_len_synopsis = curr_len_synopsis
    
print(f"{max_len_movie_name=}")
print(f"{max_len_synopsis=}")

max_len_movie_name=180
max_len_synopsis=400


In [30]:
max_len_movie_name = 0
max_len_synopsis = 0
for i in range(len(movie_dataset["test"])):
    curr_len_movie_name = len(movie_dataset["test"][i]["movie_name"])
    if curr_len_movie_name > max_len_movie_name:
        max_len_movie_name = curr_len_movie_name
        
    curr_len_synopsis = len( movie_dataset["test"][i]["synopsis"])
    if curr_len_synopsis > max_len_synopsis:
        max_len_synopsis = curr_len_synopsis
    
print(f"{max_len_movie_name=}")
print(f"{max_len_synopsis=}")

max_len_movie_name=100
max_len_synopsis=400


In [31]:
movie_dataset["train"][:3]

{'id': [44978, 50185, 34131],
 'movie_name': ['Super Me',
  'Entity Project',
  'Behavioral Family Therapy for Serious Psychiatric Disorders'],
 'synopsis': ['A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.',
  'A director and her friends renting a haunted house to capture paranormal events in order to prove it and become popular.',
  'This is an educational video for families and family therapists that describes the Behavioral Family Therapy approach to dealing with serious psychiatric illnesses.'],
 'genre': ['fantasy', 'horror', 'family']}

In [39]:
movie_dataset["train"]

Dataset({
    features: ['id', 'movie_name', 'synopsis', 'genre'],
    num_rows: 54000
})

In [59]:
dataset = movie_dataset["train"]
dataset = dataset.map(lambda example: {'text': (example['movie_name'] + ' is a movie about ' + example['synopsis']).lower()},
                      remove_columns=['movie_name', 'synopsis'])
dataset = dataset.class_encode_column("genre")
dataset

Map:   0%|          | 0/54000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/54000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'genre', 'text'],
    num_rows: 54000
})

In [60]:
dataset.features

{'id': Value(dtype='int64', id=None),
 'genre': ClassLabel(names=['action', 'adventure', 'crime', 'family', 'fantasy', 'horror', 'mystery', 'romance', 'scifi', 'thriller'], id=None),
 'text': Value(dtype='string', id=None)}

In [61]:
dataset.features

{'id': Value(dtype='int64', id=None),
 'genre': ClassLabel(names=['action', 'adventure', 'crime', 'family', 'fantasy', 'horror', 'mystery', 'romance', 'scifi', 'thriller'], id=None),
 'text': Value(dtype='string', id=None)}

In [62]:
dataset[0]

{'id': 44978,
 'genre': 4,
 'text': 'super me is a movie about a young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. selling them makes him rich.'}

In [63]:
new_dataset = dataset.train_test_split(test_size=0.2, shuffle=True,stratify_by_column="genre" )

In [64]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'genre', 'text'],
        num_rows: 43200
    })
    test: Dataset({
        features: ['id', 'genre', 'text'],
        num_rows: 10800
    })
})

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id,max_length=512)

def process(examples):
    tokenized_inputs = tokenizer(
        examples["text"], truncation=True, max_length=512
    )
    return tokenized_inputs

tokenized_datasets = new_dataset.map(process, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("genre","labels")

tokenized_datasets["train"].features

Map:   0%|          | 0/43200 [00:00<?, ? examples/s]

Map:   0%|          | 0/10800 [00:00<?, ? examples/s]

{'id': Value(dtype='int64', id=None),
 'labels': ClassLabel(names=['action', 'adventure', 'crime', 'family', 'fantasy', 'horror', 'mystery', 'romance', 'scifi', 'thriller'], id=None),
 'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [66]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from huggingface_hub import HfFolder

# create label2id, id2label dicts for nice outputs for the model
labels = tokenized_datasets["train"].features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# define data_collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [68]:
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments(
    output_dir="bert-medium-movie-genre",
    num_train_epochs=9,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    fp16=True,
    learning_rate=1e-5,
    warmup_ratio = 0.1,
    weight_decay = 0.01,
    seed=39,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id)

Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

In [69]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [70]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [71]:
def hp_space(trial):
    return {
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 5),
        "learning_rate": trial.suggest_float("learning_rate", 9e-7, 1e-3 ,log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-3, 0.1 ,log=True),
      }

In [73]:
def student_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_id,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id
    )

trainer = Trainer(
    model_init=student_init,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

best_run = trainer.hyperparameter_search(
    n_trials=30,
    direction="maximize",
    hp_space=hp_space
)

print(best_run)

Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.784531,0.370093
2,1.970500,1.713310,0.390278
3,1.712600,1.697886,0.396574


[I 2023-06-12 22:01:27,860] Trial 0 finished with value: 0.3965740740740741 and parameters: {'num_train_epochs': 3, 'learning_rate': 1.0242387553712912e-05, 'weight_decay': 0.036313308165249664}. Best is trial 0 with value: 0.3965740740740741.
Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequen

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.759483,0.378056
2,1.953800,1.695339,0.396944
3,1.681800,1.678292,0.400833
4,1.681800,1.672726,0.403889


[I 2023-06-12 22:15:31,069] Trial 1 finished with value: 0.4038888888888889 and parameters: {'num_train_epochs': 4, 'learning_rate': 1.2291045945582096e-05, 'weight_decay': 0.002878582683692837}. Best is trial 1 with value: 0.4038888888888889.
Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequen

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.316318,0.100000
2,2.190200,2.312064,0.100000
3,2.309900,2.306250,0.100000
4,2.309900,2.303125,0.100000
5,2.304700,2.302734,0.100000


[I 2023-06-12 22:33:05,954] Trial 2 finished with value: 0.1 and parameters: {'num_train_epochs': 5, 'learning_rate': 0.0008063447359961329, 'weight_decay': 0.001384129351537745}. Best is trial 1 with value: 0.4038888888888889.
Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.195664,0.214352


[W 2023-06-12 22:36:42,617] Trial 3 failed with parameters: {'num_train_epochs': 3, 'learning_rate': 1.916461608255736e-06, 'weight_decay': 0.003505380632881567} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\hazqe\AAAPython\Anaconda\envs\ai_triage\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\hazqe\AAAPython\Anaconda\envs\ai_triage\lib\site-packages\transformers\integrations.py", line 187, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "C:\Users\hazqe\AAAPython\Anaconda\envs\ai_triage\lib\site-packages\transformers\trainer.py", line 1633, in train
    return inner_training_loop(
  File "C:\Users\hazqe\AAAPython\Anaconda\envs\ai_triage\lib\site-packages\transformers\trainer.py", line 1902, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "C:\Users\hazqe\AAAPython\Anaconda\envs\ai_t

KeyboardInterrupt: 